# 인구 소멸 위기지역 - ID 부여

In [1]:
import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/mnt')

Mounted at /content/mnt


In [3]:
data_folder = '/content/mnt/MyDrive/Data/인구 소멸위기지역'

In [4]:
pop = pd.read_csv(f'{data_folder}/시군구_전처리완료.csv', encoding='euc-kr')
pop.head()

,광역시도,시군구,20-39세계,20-39세남자,20-39세여자,65세이상계,65세이상남자,65세이상여자,인구수계,인구수남자,인구수여자,소멸비율,소멸위기지역,소멸위기고위험지역
0,강원도,강릉시,47006,25199,21807,43830,18604,25227,211643,105025,106618,0.497536,True,False
1,강원도,고성군,5217,3175,2042,7755,3237,4519,26792,13864,12928,0.263314,True,False
2,강원도,동해시,18727,10469,8258,17572,7624,9949,89814,45572,44242,0.469952,True,False
3,강원도,삼척시,13583,7469,6117,16077,6766,9311,65623,33275,32348,0.380481,True,False
4,강원도,속초시,18080,9909,8171,15170,6345,8826,81497,40312,41185,0.538629,False,False


## 고유 ID 만들기

In [5]:
print(len(pop))
print(len(pop.시군구.unique()))
pop.시군구.unique()

250
226


array(['강릉시', '고성군', '동해시', '삼척시', '속초시', '양구군', '양양군', '영월군', '원주시',
       '인제군', '정선군', '철원군', '춘천시', '태백시', '평창군', '홍천군', '화천군', '횡성군',
       '가평군', '과천시', '광명시', '광주시', '구리시', '군포시', '권선구', '기흥구', '김포시',
       '남양주시', '단원구', '덕양구', '동두천시', '동안구', '만안구', '부천시', '분당구', '상록구',
       '수정구', '수지구', '시흥시', '안성시', '양주시', '양평군', '여주시', '연천군', '영통구',
       '오산시', '의왕시', '의정부시', '이천시', '일산동구', '일산서구', '장안구', '중원구', '처인구',
       '파주시', '팔달구', '평택시', '포천시', '하남시', '화성시', '거제시', '거창군', '김해시',
       '남해군', '마산합포구', '마산회원구', '밀양시', '사천시', '산청군', '성산구', '양산시', '의령군',
       '의창구', '진주시', '진해구', '창녕군', '통영시', '하동군', '함안군', '함양군', '합천군',
       '경산시', '경주시', '고령군', '구미시', '군위군', '김천시', '남구', '문경시', '봉화군', '북구',
       '상주시', '성주군', '안동시', '영덕군', '영양군', '영주시', '영천시', '예천군', '울릉군',
       '울진군', '의성군', '청도군', '청송군', '칠곡군', '광산구', '동구', '서구', '달서구', '달성군',
       '수성구', '중구', '대덕구', '유성구', '강서구', '금정구', '기장군', '동래구', '부산진구',
       '사상구', '사하구', '수영구', '연제구', '영도구', '해운대구', '강남구', '강동구', '강북구',


In [6]:
# 고성군 - 고성(강원), 고성(경남)
# 광역시 - 서울 용산, 서울 서대문, 대전 서구, 대전 유성 세종
# 행정구 - 수원 장안, 용인 수지, 고양 일산동, 창원 합포, 창원 회원 
tmp_gu_dict = {
    '수원':['장안구', '권선구', '팔달구', '영통구'], 
    '성남':['수정구', '중원구', '분당구'], 
    '안양':['만안구', '동안구'], 
    '안산':['상록구', '단원구'], 
    '고양':['덕양구', '일산동구', '일산서구'], 
    '용인':['처인구', '기흥구', '수지구'], 
    '청주':['상당구', '서원구', '흥덕구', '청원구'], 
    '천안':['동남구', '서북구'], 
    '전주':['완산구', '덕진구'], 
    '포항':['남구', '북구'], 
    '창원':['의창구', '성산구', '진해구', '마산합포구', '마산회원구']
}

In [7]:
metro_list =['서울특별시', '부산광역시', '대구광역시', '인천광역시', '대전광역시', '광주광역시', '울산광역시']
si_name = [None] * len(pop)

for i in pop.index:
    if pop.광역시도[i] in metro_list:
        if len(pop.시군구[i]) == 2:
            si_name[i] = pop.광역시도[i][:2] + ' ' + pop.시군구[i]
        else:
            si_name[i] = pop.광역시도[i][:2] + ' ' + pop.시군구[i][:-1]     # 긴 구 이름에서 '구' 제외
    else:
        if pop.시군구[i][:-1] == '고성':
            if pop.광역시도[i] == '강원도':
                si_name[i] = '고성(강원)'
            else:
                si_name[i] = '고성(경남)'
        else:
            si_name[i] = pop.시군구[i][:-1]

        for key, values in tmp_gu_dict.items():
            if pop.시군구[i] in values:
                if len(pop.시군구[i]) == 2:
                    si_name[i] = key + ' ' + pop.시군구[i]
                elif pop.시군구[i] in ['마산합포구', '마산회원구']:
                    si_name[i] = key + ' ' + pop.시군구[i][2:-1]
                else:
                    si_name[i] = key + ' ' + pop.시군구[i][:-1]

In [8]:
print(si_name)

['강릉', '고성(강원)', '동해', '삼척', '속초', '양구', '양양', '영월', '원주', '인제', '정선', '철원', '춘천', '태백', '평창', '홍천', '화천', '횡성', '가평', '과천', '광명', '광주', '구리', '군포', '수원 권선', '용인 기흥', '김포', '남양주', '안산 단원', '고양 덕양', '동두천', '안양 동안', '안양 만안', '부천', '성남 분당', '안산 상록', '성남 수정', '용인 수지', '시흥', '안성', '양주', '양평', '여주', '연천', '수원 영통', '오산', '의왕', '의정부', '이천', '고양 일산동', '고양 일산서', '수원 장안', '성남 중원', '용인 처인', '파주', '수원 팔달', '평택', '포천', '하남', '화성', '거제', '거창', '고성(경남)', '김해', '남해', '창원 합포', '창원 회원', '밀양', '사천', '산청', '창원 성산', '양산', '의령', '창원 의창', '진주', '창원 진해', '창녕', '통영', '하동', '함안', '함양', '합천', '경산', '경주', '고령', '구미', '군위', '김천', '포항 남구', '문경', '봉화', '포항 북구', '상주', '성주', '안동', '영덕', '영양', '영주', '영천', '예천', '울릉', '울진', '의성', '청도', '청송', '칠곡', '광주 광산', '광주 남구', '광주 동구', '광주 북구', '광주 서구', '대구 남구', '대구 달서', '대구 달성', '대구 동구', '대구 북구', '대구 서구', '대구 수성', '대구 중구', '대전 대덕', '대전 동구', '대전 서구', '대전 유성', '대전 중구', '부산 강서', '부산 금정', '부산 기장', '부산 남구', '부산 동구', '부산 동래', '부산 부산진', '부산 북구', '부산 사상', '부산 사하', '부산 서구', '부산 수영', '부산 연제'

In [9]:
pop['ID'] = si_name
del pop['20-39세남자']
del pop['65세이상남자']
del pop['65세이상여자']
print(pop.shape)
pop.head()

(250, 12)


,광역시도,시군구,20-39세계,20-39세여자,65세이상계,인구수계,인구수남자,인구수여자,소멸비율,소멸위기지역,소멸위기고위험지역,ID
0,강원도,강릉시,47006,21807,43830,211643,105025,106618,0.497536,True,False,강릉
1,강원도,고성군,5217,2042,7755,26792,13864,12928,0.263314,True,False,고성(강원)
2,강원도,동해시,18727,8258,17572,89814,45572,44242,0.469952,True,False,동해
3,강원도,삼척시,13583,6117,16077,65623,33275,32348,0.380481,True,False,삼척
4,강원도,속초시,18080,8171,15170,81497,40312,41185,0.538629,False,False,속초


In [10]:
pop.to_csv(f'{data_folder}/시군구_ID부여.csv', encoding='euc-kr', index=False)